In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle("WSBSubmissonsdataActual.pkl")

We make an outputs dataframe, and set a "principal" score, which is the principal axis from the PCA on these 3 features.

In [4]:
outputs = df[['score', 'num_comments', 'upvote_ratio']].astype(float)

In [5]:
outputs["norm_score"] = (outputs.score - outputs.score.mean()) / outputs.score.std()
outputs["norm_comments"] = (outputs.num_comments - outputs.num_comments.mean()) / outputs.num_comments.std()
outputs["norm_ratio"] = (outputs.upvote_ratio - outputs.upvote_ratio.mean()) / outputs.upvote_ratio.std()

In [6]:
outputs["principal"] = outputs.norm_score * 0.66154162 + outputs.norm_comments * 0.60287027 + outputs.norm_ratio * 0.44599341

In [8]:
df["principal"] = outputs.principal

We shrink the time values because they give my computer a headache.

In [9]:
df['small_times'] = df.time_created/1000000

We regress against time, because we don't want to be biased towards older posts.

In [10]:
from sklearn.linear_model import LinearRegression

slr = LinearRegression(copy_X = True)

slr.fit(df['small_times'].values.reshape(-1,1), df['principal'].values)

LinearRegression()

Our final output variable is called popularity_score.

In [12]:
b0 = slr.intercept_
b1 = slr.coef_[0]

df['popularity_score'] = df.principal - b0 - b1 * df.small_times